In [1]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.

In [2]:
from datasets import load_dataset
from google.colab import drive
import pandas as pd
import json


# Mount Google Drive and define file path (assuming it's the same as before)
drive.mount('/content/drive')
file_path = "/content/drive/MyDrive/Datasets/LLMs/MIMIC/extracted_symptoms_new.jsonl"



# ================== STEP 1: Load JSONL dataset ==================
def load_jsonl(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            item = json.loads(line.strip())
            data.append(item)
    return data

data = load_jsonl(file_path)  # replace with your dataset path
df = pd.DataFrame(data)






Mounted at /content/drive


In [3]:
print(df.shape)
print(df.columns)

(1000, 3)
Index(['instruction', 'input', 'output'], dtype='object')


In [4]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: Tesla T4


In [5]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/mistral-7b-bnb-4bit"

max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.6: Fast Mistral patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [6]:
# from datasets import Dataset

# print(ds['train'][0])

# def format_prompt(example):
#     return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

# formatted_data = [format_prompt(item) for item in file]
# dataset = Dataset.from_dict({"text": formatted_data})

In [7]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2025.9.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
from sklearn.model_selection import train_test_split

def reformat_data(example):
  """Reformats a dataset example into the desired JSON structure."""
  return {
      "instruction": example["input"],
      "input": "",
      "output": example["output"]
  }

# Split the dataframe into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42) # 70% train, 30% temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42) # 15% val, 15% test

# Convert back to list of dictionaries for reformatting
train_data = train_df.to_dict('records')
val_data = val_df.to_dict('records')
test_data = test_df.to_dict('records')


# Apply the reformatting function to the training split
reformatted_train_data = [reformat_data(example) for example in train_data]
reformatted_val_data = [reformat_data(example) for example in val_data]
reformatted_test_data = [reformat_data(example) for example in test_data]


# Display the first reformatted example to verify
print("Reformatted Training Data Example:")
print(reformatted_train_data[1])
print("\nReformatted Validation Data Example:")
print(reformatted_val_data[1])
print("\nReformatted Test Data Example:")
print(reformatted_test_data[1])

Reformatted Training Data Example:
{'instruction': 'Present Symptoms: emesis, melena, hypotensive, tachycardic, firm upper abdomen, hypotension, tachycardia, large amount of free fluid in the abdomen and pelvis, increase in the fluid collection in the left subphrenic space, small amount of high attenuation dependent material in the left flank and in the pelvis', 'input': '', 'output': 'Unspecified disorder of kidney and ureter, Personal history of tobacco use, Cyst and pseudocyst of pancreas, Diverticulosis of colon (without mention of hemorrhage), Unspecified essential hypertension, Chronic pancreatitis, Hemorrhage, unspecified, Hemoperitoneum (nontraumatic), Esophageal reflux'}

Reformatted Validation Data Example:
{'instruction': 'Present Symptoms: acute abdominal pain, midabdominal pain, dull pain, constant pain, emesis', 'input': '', 'output': 'Pure hypercholesterolemia, Unspecified essential hypertension, Aortic valve disorders, Unspecified acquired hypothyroidism, Thoracoabdomin

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import Dataset # Import Dataset

# Define the formatting function
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Apply a standard instruction tuning template
        text = f"### Instruction:\n{instruction}\n### Input:\n{input}\n### Output:\n{output}<|endoftext|>"
        texts.append(text)
    # Workaround: Return a dummy string if the list is empty to prevent IndexError during trainer validation
    if not texts:
        return ["<dummy_text>"]
    return texts # Return the list of strings

# Convert reformatted data lists to Dataset objects
train_dataset = Dataset.from_list(reformatted_train_data)
val_dataset = Dataset.from_list(reformatted_val_data)
test_dataset = Dataset.from_list(reformatted_test_data)


# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset, # Use the Dataset object
    formatting_func=formatting_prompts_func, # Pass the formatting function here
    max_seq_length=max_seq_length,
    dataset_num_proc=1, # Changed from 2 to 1
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/700 [00:00<?, ? examples/s]

In [10]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 700 | Num Epochs = 3 | Total steps = 264
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 7,409,504,256 (2.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,1.254400
50,0.916300
75,0.795900
100,0.681300
125,0.592600
150,0.576400
175,0.558700
200,0.361600
225,0.320400
250,0.327100


In [11]:
# Test the fine-tuned model
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# User instruction input
user_instruction = "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."

# Format the input according to the training template
# We only provide the instruction and input fields for inference
prompt = f"### Instruction:\n{user_instruction}\n### Input:\n"

# Tokenize the formatted prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
)

# Decode and print the full response
response = tokenizer.batch_decode(outputs)[0]
print(response)

# Optionally, extract just the generated output part
# Assuming the model generates "### Output:\n..." after the prompt
output_start_index = response.find("### Output:\n")
if output_start_index != -1:
    generated_output = response[output_start_index + len("### Output:\n"):].strip()
    print("\nGenerated Output:")
    print(generated_output)
else:
    print("\nCould not find '### Output:' in the generated response.")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<s> ### Instruction:
I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak.
### Input:

### Output:
Spinal stenosis, lumbar region, without neurogenic claudication, Cervical spondylosis with myelopathy, Unspecified essential hypertension, Coronary atherosclerosis of native coronary artery, Aortic valve disorders, Atrioventricular block, complete, Cerebral embolism with cerebral infarction, Unspecified acquired hypothyroidism<|endoftext|>|endoftext|>
### Input:

### Output:
Congestive heart failure, unspecified, Pneumonia, organism unspecified, Other and unspecified angina pectoris, Osteoporosis, unspecified, Asthma, unspecified type, unspecified<|endoftext|>|endoftext|>
### Input:
Other and unspecified angina pectoris, Pure hypercholesterolemia<|endoftext|>|endoftext|>
### Output:
Coronary atherosclerosis of native coronary artery, Unspecified essential hypertension<

Generated

In [12]:
# test=ds['test']

In [13]:
# model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

In [14]:
# from google.colab import files
# import os

# gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
# if gguf_files:
#     gguf_file = os.path.join("gguf_model", gguf_files[0])
#     print(f"Downloading: {gguf_file}")
#     files.download(gguf_file)

## Reformat test data

### Subtask:
Apply the same reformatting function used for training data to the test data to get it into the `instruction`, `input`, `output` dictionary format.

**Reasoning**:
Apply the reformatting function to the test data and remove the original columns.

In [15]:
# reformatted_test_data = test_data.map(reformat_data, remove_columns=["input", "output"])
# print(reformatted_test_data[0])

## Prepare for inference

### Subtask:
Iterate through the reformatted test data. For each example, create a prompt string using the same template as used for training and inference (`### Instruction:
{instruction}
### Input:
`).

**Reasoning**:
Initialize an empty list and iterate through the reformatted test data to create and append the formatted prompt strings to the list.

In [16]:
prompts = []
for example in reformatted_test_data:
    instruction = example["instruction"]
    input_text = example["input"]
    prompt = f"### Instruction:\n{instruction}\n### Input:\n{input_text}"
    prompts.append(prompt)

print(prompts[0])

### Instruction:
Present Symptoms:
### Input:



## Run inference

### Subtask:
For each prepared prompt, run inference using the fine-tuned model to generate the predicted diagnosis.

**Reasoning**:
Iterate through the prepared prompts, tokenize each one, generate a response using the fine-tuned model, decode the response, and store it in a list.

In [17]:
predicted_outputs = []
for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        input_ids=inputs.input_ids,
        max_new_tokens=256,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )
    response = tokenizer.batch_decode(outputs)[0]
    predicted_outputs.append(response)

print("First few predicted outputs:")
for i in range(min(5, len(predicted_outputs))):
    print(f"--- Output {i+1} ---")
    print(predicted_outputs[i])

First few predicted outputs:
--- Output 1 ---
<s> ### Instruction:
Present Symptoms:
### Input:

### Output:
Malignant neoplasm of cardia, Atrial fibrillation, Aortic valve disorders, Esophageal reflux, Gout, unspecified, Pneumonia, organism unspecified, Unspecified essential hypertension<|endoftext|>
### Input:

### Output:
Candidiasis of other urogenital sites, Unspecified acquired hypothyroidism, Aortic valve disorders, Atrial fibrillation, Esophageal reflux, Gout, unspecified, Pure hypercholesterolemia<|endoftext|>
### Input:

### Output:
Unspecified essential hypertension, Aortic valve disorders, Atrial fibrillation, Esophageal reflux<|endoftext|>
### Input:

### Output:
Gout, unspecified, Pneumonia, organism unspecified<|endoftext|>
### Input:

### Output:
Pure hypercholesterolemia, Coronary atherosclerosis of native coronary ar
--- Output 2 ---
<s> ### Instruction:
Present Symptoms: pain, emesis, diarrhea, temperature of 101 degrees fahrenheit, aspiration pneumonia, painless hem

**Reasoning**:
The model has generated responses for each prompt. The next step is to extract the actual predicted diagnosis from these generated strings, as the raw output includes the input prompt and extra tokens.

In [18]:
extracted_predictions = []
for response in predicted_outputs:
    # Find the start of the '### Output:' section
    output_start_index = response.find("### Output:\n")
    if output_start_index != -1:
        # Extract the part after '### Output:'
        generated_output = response[output_start_index + len("### Output:\n"):].strip()
        # Remove the '<|endoftext|>' token if present
        if generated_output.endswith("<|endoftext|>"):
            generated_output = generated_output[:-len("<|endoftext|>")].strip()

        # Split the string by commas and strip whitespace to get a list of diagnoses
        diagnoses_list = [diag.strip() for diag in generated_output.split(',') if diag.strip()]
        extracted_predictions.append(diagnoses_list)
    else:
        # If '### Output:' is not found, append an empty list
        extracted_predictions.append([])

print("First few extracted predictions:")
for i in range(min(5, len(extracted_predictions))):
    print(f"--- Prediction {i+1} ---")
    print(extracted_predictions[i])

First few extracted predictions:
--- Prediction 1 ---
['Malignant neoplasm of cardia', 'Atrial fibrillation', 'Aortic valve disorders', 'Esophageal reflux', 'Gout', 'unspecified', 'Pneumonia', 'organism unspecified', 'Unspecified essential hypertension<|endoftext|>\n### Input:\n\n### Output:\nCandidiasis of other urogenital sites', 'Unspecified acquired hypothyroidism', 'Aortic valve disorders', 'Atrial fibrillation', 'Esophageal reflux', 'Gout', 'unspecified', 'Pure hypercholesterolemia<|endoftext|>\n### Input:\n\n### Output:\nUnspecified essential hypertension', 'Aortic valve disorders', 'Atrial fibrillation', 'Esophageal reflux<|endoftext|>\n### Input:\n\n### Output:\nGout', 'unspecified', 'Pneumonia', 'organism unspecified<|endoftext|>\n### Input:\n\n### Output:\nPure hypercholesterolemia', 'Coronary atherosclerosis of native coronary ar']
--- Prediction 2 ---
['Subendocardial infarction', 'initial episode of care', 'Chronic airway obstruction', 'not elsewhere classified', 'Pneumon

**Reasoning**:
The predicted diagnoses have been extracted. The next step is to get the true diagnoses from the reformatted test data to compare them.

In [19]:
true_diagnoses = []
for example in reformatted_test_data:
    # Split the string by commas and strip whitespace to get a list of diagnoses
    diagnoses_list = [diag.strip() for diag in example["output"].split(',') if diag.strip()]
    true_diagnoses.append(diagnoses_list)

print("First few true diagnoses:")
for i in range(min(5, len(true_diagnoses))):
    print(f"--- True Diagnosis {i+1} ---")
    print(true_diagnoses[i])

First few true diagnoses:
--- True Diagnosis 1 ---
['Chronic glomerulonephritis in diseases classified elsewhere', 'Other specified surgical operations and procedures causing abnormal patient reaction', 'or later complication', 'without mention of misadventure at time of operation', 'Accidents occurring in residential institution', 'Other iatrogenic hypotension', 'Long QT syndrome', 'End stage renal disease', 'Hypertensive chronic kidney disease', 'unspecified', 'with chronic kidney disease stage V or end stage renal disease', 'Disorders of magnesium metabolism', 'Paroxysmal ventricular tachycardia', 'Acidosis', 'Pneumonia', 'organism unspecified', 'Atrial flutter', 'Atrial fibrillation', 'Ventricular fibrillation', 'Cardiac arrest', 'Cardiac complications', 'not elsewhere classified', 'nan', 'Systemic lupus erythematosus', 'Hyposmolality and/or hyponatremia', 'Secondary hyperparathyroidism (of renal origin)']
--- True Diagnosis 2 ---
['Subendocardial infarction', 'initial episode of c

## Evaluate model

### Subtask:
Compare the list of generated outputs with the list of true outputs to calculate evaluation metrics such as accuracy.

**Reasoning**:
Calculate the accuracy of the model's predictions by comparing the extracted predictions with the true diagnoses and print the result.

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from sklearn.preprocessing import MultiLabelBinarizer

# Calculate Accuracy
# Note: Accuracy is calculated here assuming exact match of the list of diagnoses
# This might be too strict for multi-label classification.
# A more appropriate metric might be average Jaccard similarity or F1 score.
# For demonstration, we calculate accuracy based on exact list match.

# Use MultiLabelBinarizer to convert lists of diagnoses to binary format
mlb = MultiLabelBinarizer()
true_diagnoses_bin = mlb.fit_transform(true_diagnoses)
extracted_predictions_bin = mlb.transform(extracted_predictions)


# Calculate accuracy using the binary format
# For multi-label accuracy, we typically look at metrics like subset accuracy or Hamming loss.
# accuracy_score with normalize=True (default) calculates subset accuracy (all labels must match).
accuracy = accuracy_score(true_diagnoses_bin, extracted_predictions_bin)
print(f"Accuracy (subset accuracy): {accuracy}")

# Calculate Jaccard Similarity for each example and then average
jaccard_scores = []
for true_list, predicted_list in zip(true_diagnoses, extracted_predictions):
    # Convert lists to sets for easy intersection and union calculation
    true_set = set(true_list)
    predicted_set = set(predicted_list)

    # Calculate intersection and union
    intersection = len(true_set.intersection(predicted_set))
    union = len(true_set.union(predicted_set))

    # Avoid division by zero if both sets are empty
    if union == 0:
        jaccard = 1.0  # Or 0.0, depending on how you want to handle this case
    else:
        jaccard = intersection / union
    jaccard_scores.append(jaccard)

# Calculate the average Jaccard similarity
average_jaccard = sum(jaccard_scores) / len(jaccard_scores) if jaccard_scores else 0.0
print(f"Average Jaccard Similarity: {average_jaccard}")

Accuracy (subset accuracy): 0.0
Average Jaccard Similarity: 0.10988379500400694


/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['000-1', '000-2', '249 grams', '25-26 completed weeks of gestation', '25-26 completed weeks of gestation<|endoftext|>\n### Input:\n\n### Output:\nExtreme immaturity', '25-26 completed weeks of gestation<|endoftext|>\n### Input:\n\n### Output:\nIntraventricular hemorrhage', '25-26 completed weeks of gestation<|endoftext|>\n### Input:\n\n### Output:\nRespiratory distress syndrome in newborn', '25-26 completed weeks of gestation<|endoftext|>|endoftext|>\n### Input:\n\n### Output:\nTwin birth', '250-1', '27-28 completed weeks of gestation<|endoftext|>\n### Input:', '27-28 completed weeks of gestation<|endoftext|>|endoftext|>\n### Input:\n\n### Output:\nObservation for suspected infectious condition', '27-28 completed weeks of gestation<|endoftext|>|endoftext|>\n### Input:\n\n### Output:\nSepticemia [sepsis] of newborn', '33-34 completed weeks of gestation<|endoftext|>\n### Output:\n

## Display results

### Subtask:
Print or display the evaluation results.

**Reasoning**:
Print the calculated accuracy score and the number of test examples evaluated as requested by the subtask instructions.

In [21]:
print(f"Accuracy: {accuracy}")
print(f"Number of test examples evaluated: {len(true_diagnoses)}")

Accuracy: 0.0
Number of test examples evaluated: 150


In [22]:
average_jaccard_val = sum(jaccard_scores_val) / len(jaccard_scores_val) if jaccard_scores else 0.0
print(f"Average Jaccard Similarity: {average_jaccard_val}")

NameError: name 'jaccard_scores_val' is not defined

In [ ]:
jaccard_scores_val=151.88
jaccard_scores_val=[2,3]